--- Day 8: Treetop Tree House ---

The expedition comes across a peculiar patch of tall trees all planted carefully in a grid. The Elves explain that a previous expedition planted these trees as a reforestation effort. Now, they're curious if this would be a good location for a tree house.

First, determine whether there is enough tree cover here to keep a tree house hidden. To do this, you need to count the number of trees that are visible from outside the grid when looking directly along a row or column.

The Elves have already launched a quadcopter to generate a map with the height of each tree (your puzzle input). For example:
```
30373
25512
65332
33549
35390
```
Each tree is represented as a single digit whose value is its height, where 0 is the shortest and 9 is the tallest.

A tree is visible if all of the other trees between it and an edge of the grid are shorter than it. Only consider trees in the same row or column; that is, only look up, down, left, or right from any given tree.

All of the trees around the edge of the grid are visible - since they are already on the edge, there are no trees to block the view. In this example, that only leaves the interior nine trees to consider:

    The top-left 5 is visible from the left and top. (It isn't visible from the right or bottom since other trees of height 5 are in the way.)
    The top-middle 5 is visible from the top and right.
    The top-right 1 is not visible from any direction; for it to be visible, there would need to only be trees of height 0 between it and an edge.
    The left-middle 5 is visible, but only from the right.
    The center 3 is not visible from any direction; for it to be visible, there would need to be only trees of at most height 2 between it and an edge.
    The right-middle 3 is visible from the right.
    In the bottom row, the middle 5 is visible, but the 3 and 4 are not.

With 16 trees visible on the edge and another 5 visible in the interior, a total of 21 trees are visible in this arrangement.

Consider your map; how many trees are visible from outside the grid?


In [158]:
public record struct Position {
    public int X;
    public int Y;
}

Dictionary<Position, int> ParseTrees(IEnumerable<string> input) {
    return input.SelectMany((row, y) => row.ToCharArray()
        .Select((height, x) => new { Position = new Position { X = x, Y = y }, Height = int.Parse($"{height}") })
    ).ToDictionary(tree => tree.Position, tree => tree.Height);
}

bool IsVisible(KeyValuePair<Position, int> tree, Dictionary<Position, int> trees) {
    var position = tree.Key;
    var height = tree.Value;
    var higherTrees = trees.Where(other => other.Value >= height);
    var topTrees = higherTrees.Where(other => position.X == other.Key.X && position.Y > other.Key.Y);
    if (!topTrees.Any()) {
        count += 1;
        return true;
    }
    var bottomTrees = higherTrees.Where(other => position.X == other.Key.X && position.Y < other.Key.Y);
    if (!bottomTrees.Any()) {
        count += 1;
        return true;
    }
    var leftTrees = higherTrees.Where(other => position.Y == other.Key.Y && position.X > other.Key.X);
    if (!leftTrees.Any()) {
        count += 1;
        return true;
    }
    var rightTrees = higherTrees.Where(other => position.Y == other.Key.Y && position.X < other.Key.X);
    if (!rightTrees.Any()) {
        count += 1;
        return true;
    }
    return false;
}

int ScenicScore(KeyValuePair<Position, int> tree, Dictionary<Position, int> trees, int maxWidth, int maxHeight) {
    var position = tree.Key;
    var height = tree.Value;
    var higherTrees = trees.Where(other => other.Value >= height);
    var viewingScore = 1;

    var topTrees = higherTrees.Where(other => position.X == other.Key.X && position.Y > other.Key.Y)
        .OrderByDescending(other => other.Key.Y);
    if (topTrees.Any()) {
        var diff = position.Y - topTrees.First().Key.Y;
        viewingScore = viewingScore * diff;
    } else {
        viewingScore = viewingScore * position.Y;
    }

    var bottomTrees = higherTrees.Where(other => position.X == other.Key.X && position.Y < other.Key.Y)
        .OrderBy(other => other.Key.Y);;
    if (bottomTrees.Any()) {
        var diff = bottomTrees.First().Key.Y - position.Y;
        viewingScore = viewingScore * diff;
    } else {
        viewingScore = viewingScore * (maxHeight - position.Y);
    }

    var leftTrees = higherTrees.Where(other => position.Y == other.Key.Y && position.X > other.Key.X)
        .OrderByDescending(other => other.Key.X);;
    if (leftTrees.Any()) {
        var diff = position.X - leftTrees.First().Key.X;
        viewingScore = viewingScore * diff;
    } else {
        viewingScore = viewingScore * position.X;
    }

    var rightTrees = higherTrees.Where(other => position.Y == other.Key.Y && position.X < other.Key.X)
        .OrderBy(other => other.Key.X);
    if (rightTrees.Any()) {
        var diff = rightTrees.First().Key.X - position.X;
        viewingScore = viewingScore * diff;
    } else {
        viewingScore = viewingScore * (maxWidth - position.X);
    }
    
    return viewingScore;
}

In [159]:
var input = System.IO.File.ReadAllLines(@"input.txt");
input


index,value
0,122102002221220113213120331300324403204320144533424135412222404323003104143241322330203332311001201
1,122101021201100333103013020111134433225413441345223355455242454420324041424144143320331213313011211
2,002020102202133322040121034302303042143134335334145322255225234524340004204432223232112102301322020
3,220012312123310100132111431130112432541411111551552451554212423331432121222413114341113030121331012
4,201002212201312002313134042122454351541442441552442121522445315151245342234433410040022122332121102
5,022032110101034413122002140132134344355151432255122144512123321252231413241444212332310003201030321
6,202221121312021412211244435533212115121444135555133441115332134551132345451112300141040402100300202
7,012312220202241130000011113153354512345554455246352624652351411552522235252320220001241023300202002
8,012333201303122341124245121245445543543226522355634323552563215553344215343251121231221244211102101
9,321301231011002302232325451215424531566442635526336536256234645441454222315444511024423430030333330


In [160]:
var trees = ParseTrees(input);
var count = 0;
trees.Where(tree => IsVisible(tree, trees)).Count()

1812

In [161]:
var trees = ParseTrees(input);
var count = 0;
// ScenicScore(new KeyValuePair<Position, int>(new Position { X = 2, Y = 3}, 5), trees, 4, 4)
var maxWidth = trees.Select(tree => tree.Key.X).Max();
var maxHeight = trees.Select(tree => tree.Key.Y).Max();
trees.Select(tree => ScenicScore(tree, trees, maxWidth, maxHeight)).Max()


315495